In [86]:
import pandas as pd
import urllib.request, json 
from urllib.error import HTTPError

In [6]:
with urllib.request.urlopen("https://codex.opendata.api.vlaanderen.be:443/api/Trefwoord") as url:
    print("df")
    data = json.load(url)
    print(data)

df
{'TotaalAantal': 2314, 'ResultatenLijst': [{'Naam': 'taalgebruik', 'Link': {'Rel': 'self', 'Href': 'https://codex.opendata.api.vlaanderen.be/api/Trefwoord/1000001'}, 'Id': 1000001}, {'Naam': 'Promotie Landbouw, Tuinbouw en Zeevisserijproducten vzw.', 'Link': {'Rel': 'self', 'Href': 'https://codex.opendata.api.vlaanderen.be/api/Trefwoord/1000002'}, 'Id': 1000002}, {'Naam': 'Vlaams Fonds promotie producten landbouw/tuinbouw/zeevisserij', 'Link': {'Rel': 'self', 'Href': 'https://codex.opendata.api.vlaanderen.be/api/Trefwoord/1000003'}, 'Id': 1000003}, {'Naam': 'Vlaamse dienst voor agro-marketing', 'Link': {'Rel': 'self', 'Href': 'https://codex.opendata.api.vlaanderen.be/api/Trefwoord/1000004'}, 'Id': 1000004}, {'Naam': 'VLAM', 'Link': {'Rel': 'self', 'Href': 'https://codex.opendata.api.vlaanderen.be/api/Trefwoord/1000005'}, 'Id': 1000005}, {'Naam': 'Vlaams Promotiecentrum voor Agro- en Visserijmarketing', 'Link': {'Rel': 'self', 'Href': 'https://codex.opendata.api.vlaanderen.be/api/Tre

In [19]:
# get the IDs of all the trefworden

trefwoord_ids = []
for item in data["ResultatenLijst"]:
    trefwoord_ids.append(item["Id"])

<h1> Documents </h1>

In [96]:
# get the corresponding documents and their attributes

doc_ids = []
doc_title = []
doc_type = []
doc_date = []
for id in trefwoord_ids:
    with urllib.request.urlopen(f"https://codex.opendata.api.vlaanderen.be:443/api/Trefwoord/{id}/WetgevingDocumenten") as url:
        data = json.load(url)
        for i in data["ResultatenLijst"]:
            doc_ids.append(i["Id"])
            doc_title.append(i["Opschrift"])
            doc_type.append(i["WetgevingDocumentType"])
            doc_date.append(i["Datum"])

# 3838 documents

In [137]:
# creating doc df
df_doc =pd.DataFrame()
df_doc["id"] = doc_ids
df_doc["about"] = doc_title
df_doc["date"] = doc_date
df_doc["type"] = doc_type

df_doc["date"]= pd.to_datetime(df_doc["date"])
df_doc["year"] = df_doc["date"].dt.year
df_doc["month"] = df_doc["date"].dt.month
df_doc["day"] = df_doc["date"].dt.day

In [134]:
# creating date df
date_df = df_doc[["id", "date"]]
date_df['id'] = date_df.index
date_df.to_csv("date.csv", index = False)

C:\Users\nastj\AppData\Local\Temp\ipykernel_12028\4218067922.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df['id'] = date_df.index


<h1> Thema </h1>

In [ ]:
# https://codex.opendata.api.vlaanderen.be:443/api/WetgevingDocument/1000001/Themas
doc_thema = []
for id in doc_ids:
    with urllib.request.urlopen(f"https://codex.opendata.api.vlaanderen.be:443/api/WetgevingDocument/{id}/Themas") as url:
        data = json.load(url)
        for thema1 in data["Items"]:
            print(thema1)
            doc_thema.append(thema1["Omschrijving"])
            # for thema2 in thema1["Items"]:
            #     print(thema2)
        #     if thema:
        #         doc_thema.append(thema["Items"])
        #     else:
        #         doc_thema.append("unknown")
# print(doc_thema)

<h1> Articles </h1>

In [ ]:
# get articles ids (not the versions, just articles)
art_ids = []
for id in doc_ids:
    try:
        with urllib.request.urlopen(f"https://codex.opendata.api.vlaanderen.be:443/api/WetgevingDocument/{id}/VolledigDocument") as url:
            data = json.load(url)
            for art_id in data["Inhoud"]["Artikelen"]:
                art_ids.append(art_id["Id"])
    except urllib.error.HTTPError:
        art_ids.append("unknown")
        continue

art_ids[:5] # IncompleteRead: IncompleteRead(14040804 bytes read, 4331923 more expected) ; 18292

<h1> Articles version </h1>

In [146]:
art_v_ids = []
for id in art_ids:
    try:
        with urllib.request.urlopen(f"https://codex.opendata.api.vlaanderen.be:443/api/WetgevingArtikel/{id}/Versies") as url:
            data = json.load(url)
            if data["TotaalAantal"] > 1:
                art_v_ids.append("no versions")
            else:
                list_v_ids = []
                for i in data["ResultatenLijst"]: 
                    list_v_ids.append(i["Id"])
                art_v_ids.append(list_v_ids)

    except urllib.error.HTTPError:
        art_ids.append("unknown")
        continue

In [148]:
df_art_v = pd.DataFrame()
df_art_v["art_v_id"] = art_v_ids
df_art_v.to_csv("df_art_v.csv", index = False)

In [173]:
df_art_v = pd.read_csv("df_art_v.csv")

In [176]:
art_v_start_date = []
art_v_end_date = []
art_v_title = []

for id in df_art_v["art_v_id"]:
    print(id)
    if id == "no versions":
        continue
    else:
        # try:
        with urllib.request.urlopen(f"https://codex.opendata.api.vlaanderen.be:443/api/WetgevingArtikelVersie/{id}/VolledigeArtikelVersie") as url:
            data = json.load(url)
            art_v_start_date.append(data["Document"]["StartDatum"])
            art_v_end_date.append(data["Document"]["EindDatum"])
            art_v_title.append(data["Document"]["Opschrift"])

1015859
1015860
1015861
1015862
1241249
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
1016212
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
1016227
1016228
1016229
no versions
1016231
1016232
no versions
1016234
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
1016243
no versions
no versions
no versions
1016247
no versions
no versions
no versions
1016251
no versions
1016253
1016255
1016256
1016257
no versions
no versions
1016260
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions
no versions


URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [178]:
len(df_art_v["art_v_id"])

8331

In [179]:
df_art_v = df_art_v[df_art_v.art_v_id != "no versions"]
df_art_v["startdate"] = art_v_start_date
df_art_v["enddate"] = art_v_end_date
df_art_v["title"]= art_v_title
df_art_v.to_csv("df_article_versions_ids.csv", index = False)

ValueError: Length of values (3498) does not match length of index (3721)

In [ ]:
# article and article versions links
df_art_artv = pd.DataFrame()
df_art_artv["article_id"]  = art_ids
df_art_artv["article_v_id"]  = art_v_ids
df_art_artv["rel"]  = "version"
df_art_artv = df_art_artv[df_art_artv.article_v_id != "no versions"]

df_art_artv.to_csv("df_rel_art_artv.csv", index = False)